<center>
<img src="https://laelgelcpublic.s3.sa-east-1.amazonaws.com/lael_50_years_narrow_white.png.no_years.400px_96dpi.png" width="300" alt="LAEL 50 years logo">
<h3>APPLIED LINGUISTICS GRADUATE PROGRAMME (LAEL)</h3>
</center>
<hr>

# Getting started tutorial

In [1]:
from pyspark.sql import SparkSession

def calculate_red_violations(data_source, output_uri):
    """
    Processes sample food establishment inspection data and queries the data to find the top 10 establishments
    with the most Red violations from 2006 to 2020.

    :param data_source: The URI of your food establishment data CSV, such as 's3://DOC-EXAMPLE-BUCKET/food-establishment-data.csv'.
    :param output_uri: The URI where output is written, such as 's3://DOC-EXAMPLE-BUCKET/restaurant_violation_results'.
    """
    with SparkSession.builder.appName("Calculate Red Health Violations").getOrCreate() as spark:
        # Load the restaurant violation CSV data
        if data_source is not None:
            restaurants_df = spark.read.option("header", "true").csv(data_source)

        # Create an in-memory DataFrame to query
        restaurants_df.createOrReplaceTempView("restaurant_violations")

        # Create a DataFrame of the top 10 restaurants with the most Red violations
        top_red_violation_restaurants = spark.sql("""SELECT name, count(*) AS total_red_violations 
          FROM restaurant_violations 
          WHERE violation_type = 'RED' 
          GROUP BY name 
          ORDER BY total_red_violations DESC LIMIT 10""")

        # Write the results to the specified output URI
        top_red_violation_restaurants.write.option("header", "true").mode("overwrite").csv(output_uri)

# Set the values for data_source and output_uri
data_source = 's3://gelcawsemr/food_establishment_data.csv'
output_uri = 's3://gelcawsemr/restaurant_violation_results'

# Call the function with the provided values
calculate_red_violations(data_source, output_uri)


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1701743246151_0004,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Programme to uncompress the archives

In [ ]:
import boto3
import tarfile
import bz2
import os

# Set up AWS credentials
aws_access_key_id = 'YOUR_ACCESS_KEY_ID'
aws_secret_access_key = 'YOUR_SECRET_ACCESS_KEY'
region_name = 'YOUR_REGION_NAME'

# Set up S3 client
s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=region_name)

# Set up EMR client
emr = boto3.client('emr', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=region_name)

# Set up the job flow
job_flow_id = 'YOUR_JOB_FLOW_ID'

# Set up the source and destination S3 bucket names
source_bucket_name = 'YOUR_SOURCE_BUCKET_NAME'
destination_bucket_name = 'YOUR_DESTINATION_BUCKET_NAME'

# Set up the S3 bucket and file paths
tar_file_key = 'path/to/your/tar/file.tar'

# Fetch the .tar file from the source S3 bucket
s3.download_file(source_bucket_name, tar_file_key, 'file.tar')

# Uncompress the .tar file
with tarfile.open('file.tar', 'r') as tar:
    tar.extractall()

# Iterate over the extracted files
for root, dirs, files in os.walk('.'):
    for file in files:
        if file.endswith('.bz2'):
            # Uncompress each .bz2 file
            with bz2.open(os.path.join(root, file), 'rb') as bz_file:
                uncompressed_data = bz_file.read()
                # Process the uncompressed data (e.g., write to a new file, parse JSON, etc.)
                # Your processing logic goes here
                
                # Get the relative path of the file within the directory tree
                relative_path = os.path.relpath(os.path.join(root, file), '.')

                # Upload the processed file to the destination S3 bucket with the same directory tree structure
                destination_key = os.path.join(relative_path, file)
                s3.put_object(Body=uncompressed_data, Bucket=destination_bucket_name, Key=destination_key)

# Terminate the EMR cluster
emr.terminate_job_flows(JobFlowIds=[job_flow_id])

## Sample code to parse the twee

In [ ]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("Tweet Analysis") \
    .getOrCreate()

# Read the tweets file from S3 and create a dataframe
tweets_df = spark.read.json("s3://your-bucket/tweets.json")

# Filter the dataframe to extract the desired data
filtered_df = tweets_df.filter(tweets_df['text'].contains("your-filter-keyword"))

# Show the filtered dataframe
filtered_df.show()